# Toronto might not be as safe as you thought
Toronto is considered one of the safest cities in America and the 24th safest in the world behind metropolises like Tokyo, London, Paris and Seoul. Yet, historicaly, the crime rate reached alarming peaks that attracted wolds attention, specially in terms of gun violence. That is why I take the chance to explore this aspect of the city and provide the audience with a engaging look at the evolution armed crime. The dataset on hand for the analysis is part of the Toronto Police public database and can be found [here](https://data.torontopolice.on.ca/datasets/TorontoPS::shooting-and-firearm-discharges-open-data/about). With that said, let's jump into it!

## Exploratory Data Analysis (EDA)
The dataset at hand was cleaned, prepared and readied for analysis by the public servants. This means that it contains no null values to be treated in this section. As a result, the focus of the analysis will be to find variables with most utility, exclude the others and derive new ones that will increase range of the knowledge on this matter. 

In [1]:
# We'll borrow a Python funtion to read in our csv file only this once.
import pandas as pd

shootings = pd.read_csv('torontoSFD.csv')

In [2]:
-- How does the dataset look like?
SELECT *
  FROM shootings
 LIMIT 10; 

/* After a glance, some variables with similar values between them can be 
easily identified. They will be removed at a later stage of the analysis */ 

,X,Y,OBJECTID,EVENT_UNIQUE_ID,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DOW,OCC_DOY,OCC_DAY,OCC_HOUR,OCC_TIME_RANGE,DIVISION,DEATH,INJURIES,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-8.854154e+06,5.424330e+06,1,GO-2004133803,2004/01/01 05:00:00+00,2004,January,Thursday,1,1,0,Night,D31,0,1,23,Pelmo Park-Humberlea,23,Pelmo Park-Humberlea (23),-79.538218,43.733742
1,6.327780e-09,5.664924e-09,2,GO-2004120070,2004/01/03 05:00:00+00,2004,January,Saturday,3,3,1,Night,D54,0,0,NSA,NSA,NSA,NSA,0.000000,0.000000
2,-8.838674e+06,5.412379e+06,3,GO-2004135509,2004/01/04 05:00:00+00,2004,January,Sunday,4,4,4,Night,D14,0,2,78,Kensington-Chinatown,78,Kensington-Chinatown (78),-79.399163,43.656119
3,-8.822742e+06,5.424200e+06,4,GO-2004126451,2004/01/05 05:00:00+00,2004,January,Monday,5,5,13,Afternoon,D41,0,1,124,Kennedy Park,124,Kennedy Park (124),-79.256039,43.732900
4,-8.817329e+06,5.434822e+06,5,GO-2004139160,2004/01/06 05:00:00+00,2004,January,Tuesday,6,6,19,Evening,D42,0,0,146,Malvern East,132,Malvern (132),-79.207412,43.801802
5,-8.817229e+06,5.435832e+06,6,GO-2004136310,2004/01/06 05:00:00+00,2004,January,Tuesday,6,6,19,Evening,D42,0,0,146,Malvern East,132,Malvern (132),-79.206516,43.808350
6,-8.820272e+06,5.431734e+06,7,GO-2004125755,2004/01/08 05:00:00+00,2004,January,Thursday,8,8,8,Morning,D42,1,0,142,Woburn North,137,Woburn (137),-79.233852,43.781782
7,-8.850709e+06,5.421815e+06,8,GO-2004136086,2004/01/08 05:00:00+00,2004,January,Thursday,8,8,14,Afternoon,D42,1,0,28,Rustic,28,Rustic (28),-79.507268,43.717416
8,-8.818836e+06,5.435414e+06,9,GO-2004147495,2004/01/13 05:00:00+00,2004,January,Tuesday,13,13,18,Evening,D42,0,1,145,Malvern West,132,Malvern (132),-79.220952,43.805643
9,-8.840952e+06,5.433976e+06,10,GO-2004147139,2004/01/18 05:00:00+00,2004,January,Sunday,18,18,19,Evening,D32,0,1,50,Newtonbrook East,50,Newtonbrook East (50),-79.419619,43.796317


In [3]:
-- Measuring the dimensions of the dataset

SELECT (SELECT COUNT(*)
		FROM shootings) AS total_records,
		
		(SELECT COUNT(*)
		   FROM	information_schema.columns
		  WHERE table_name = 'shootings') AS total_columns;


,total_records,total_columns
0,6051,0


In [4]:
-- Summary statistics for both of the non-date variables
SELECT 'SUM' AS Statistic, SUM(injuries) AS Injuries, SUM(death) AS Deaths FROM shootings
UNION
SELECT 'AVG', ROUND(AVG(injuries),2), ROUND(AVG(death),2) FROM shootings
UNION
SELECT 'MAX', MAX(injuries), MAX(death) FROM shootings
UNION
SELECT 'MIN', MIN(injuries), MIN(injuries) FROM shootings
UNION
SELECT 'MEAN',ROUND((SUM(injuries)/COUNT(injuries)),2), ROUND((SUM(death)/COUNT(death)),2) FROM shootings

,Statistic,Injuries,Deaths
0,MEAN,0.48,0.12
1,SUM,2925.00,728.00
2,MIN,0.00,0.00
3,MAX,24.00,3.00
4,AVG,0.48,0.12


In [5]:
-- Extracting deadliest neighbourhoods based on number of casualties over the years
SELECT neighbourhood,
	   RANK() OVER (ORDER BY SUM (CASE WHEN casualties = 'Positive' 
	   							  THEN 1 ELSE 0 END) DESC) AS rank_casualties
  FROM (
  	   SELECT neighbourhood_158 AS neighbourhood,
  			  (CASE WHEN death > 0 OR injuries > 0 THEN 'Positive'
			  ELSE 'Negative' END) AS Casualties
		 FROM shootings)
 GROUP BY neighbourhood
 ORDER BY rank_casualties;
 

/* Using the RANK and SUM functions deadliest neighbourhoods, in terms of casualties, can be easily identified        amoung all the districts in Toronto. Yet, all these names aren't requiered to perform the analysis since 159    	 neighbourhood is too large. As a result, future applications of this output will be limited to the top 30       	districts which will enclose most of the occurrences. */

,neighbourhood,rank_casualties
0,Glenfield-Jane Heights,1
1,Mount Olive-Silverstone-Jamestown,2
2,Black Creek,3
3,West Humber-Clairville,4
4,Wellington Place,5
...,...,...
154,Willowdale West,149
155,Forest Hill North,149
156,Casa Loma,157
157,Forest Hill South,157


In [6]:
-- First recliclyng our previous query to limit the number of records in the final output
WITH districts AS
				(SELECT neighbourhood,
					   SUM (CASE WHEN casualties = 'Positive' 
					   THEN 1 ELSE 0 END) AS total_casualties
				  FROM (
					   SELECT neighbourhood_158 AS neighbourhood,
							  (CASE WHEN death > 0 OR injuries > 0 THEN 'Positive'
							  ELSE 'Negative' END) AS Casualties
						 FROM shootings)
				 GROUP BY neighbourhood
				 ORDER BY total_casualties DESC
				 LiMIT 30)
				 
-- Creating two new dereived variables from time-series attributes. 
SELECT objectid AS id,
	   (CASE WHEN occ_month IN ('January', 'February', 'March') THEN 'Q1'
	   		 WHEN occ_month IN ('April', 'May', 'June') THEN 'Q2'
			 WHEN occ_month IN ('July', 'August', 'September') THEN 'Q3'
			 WHEN occ_month IN ('October', 'November', 'December') THEN 'Q4'
			 ELSE NULL END) AS quarter,
	   (CASE WHEN occ_year BETWEEN 2013 AND 2023 THEN 'Current_Decade'
			 WHEN occ_year BETWEEN 2004 AND 2012 THEN 'Past_Decade'
			 ELSE NULL END) AS decade,
	   (CASE WHEN death > 0 OR injuries > 0 THEN 'Positive'
			 ELSE 'Negative' END) AS casualties
  FROM shootings AS s
  	   INNER JOIN districts AS d
	   ON s.neighbourhood_158 = d.neighbourhood;
	   
/* In this ocassion, the CASE statement does an exellent work categorizng the data to create new derived attributes. These new columns will be saved seperately to join the analytical file later. The reason behind is
to improve readability and compactness of the code within the notebook */
  


,id,quarter,decade,casualties
0,2,Q1,Past_Decade,Negative
1,3,Q1,Past_Decade,Positive
2,5,Q1,Past_Decade,Negative
3,6,Q1,Past_Decade,Negative
4,8,Q1,Past_Decade,Positive
...,...,...,...,...
3232,5424,Q2,Current_Decade,Negative
3233,5444,Q2,Current_Decade,Negative
3234,5460,Q2,Current_Decade,Negative
3235,5491,Q2,Current_Decade,Negative


In [7]:
--Time to create analytical file with the results from previous queries
WITH districs AS -- Saving the previous query as a cte to filter only the top 30 districts 
			   (SELECT neighbourhood,
					   SUM (CASE WHEN casualties = 'Positive' 
	   				   THEN 1 ELSE 0 END) AS total_casualties
				  FROM (
					   SELECT neighbourhood_158 AS neighbourhood,
							  (CASE WHEN death > 0 OR injuries > 0 THEN 'Positive'
							  ELSE 'Negative' END) AS Casualties
						 FROM shootings)
				 GROUP BY neighbourhood
				 ORDER BY total_casualties DESC
				 LiMIT 30)
				 
--Also, it will be listed and aliased the variables of interest for the analytical file		

SELECT  objectid AS id, occ_year AS year, occ_month AS month, occ_dow AS weekday, occ_time_range AS time_period,
		occ_hour AS hour, death AS death, injuries AS injuries, neighbourhood_158 AS neighbourhood, long_wgs84 AS 		  longitude, lat_wgs84 AS latitude
  FROM  shootings
 WHERE  neighbourhood_158
 	    IN (SELECT neighbourhood
			  FROM districs)
			  

,id,year,month,weekday,time_period,hour,death,injuries,neighbourhood,longitude,latitude
0,2,2004,January,Saturday,Night,1,0,0,NSA,0.000000,0.000000
1,3,2004,January,Sunday,Night,4,0,2,Kensington-Chinatown,-79.399163,43.656119
2,5,2004,January,Tuesday,Evening,19,0,0,Malvern East,-79.207412,43.801802
3,6,2004,January,Tuesday,Evening,19,0,0,Malvern East,-79.206516,43.808350
4,8,2004,January,Thursday,Afternoon,14,1,0,Rustic,-79.507268,43.717416
...,...,...,...,...,...,...,...,...,...,...,...
3232,6042,2023,December,Saturday,Evening,19,0,1,Malvern West,-79.224217,43.804488
3233,6043,2023,December,Saturday,Night,1,1,1,Humber Summit,-79.552199,43.769144
3234,6044,2023,December,Sunday,Night,2,0,2,Wellington Place,-79.391559,43.647579
3235,6049,2023,December,Thursday,Afternoon,13,0,0,York University Heights,-79.487324,43.760417


## Conclusion

The EDA helped identify repeated columns that will not have a practical use for future analysis. Generating summary statistics was useful to understand the dimmentions in both of the non-date columns in the datase. Following that, the top 30 neighbourhoods with most casualties were recognized and used to filter the final analytical files. The first will do a great grouping our records by outcome and time ranges, while the second dropped some useless variables spoted at the beginning of the analys and renamed the rest for smoother interpretability.